In [1]:
import time
import gc

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt 

from py.od_perturber import od_perturber
from py.output_reader import output_reader
from py.tap_trial import tap_trial

# Austin case study

This network represents the city of Austin, TX in the early 2000s. Suppose there was some uncertainty about how the city would grow in the next 10 years, how would that affect the output metrics and, consequently, the ranking of different investment alternatives?

In [2]:
net = 'Austin_sdb'
netFilename = 'inputs/{}_net.txt'.format(net)
tripsFilename = 'inputs/{}_trips.txt'.format(net)

ten_year_growth_factor = lambda x: x**10
ten_year_growth_factor(1.01)

1.1046221254112045

## Scenario 1: 2.08% annual growth

In [3]:
start = time.time()
df208, tstt208, wvc208, demand208, od208 = tap_trial(
    netFilename, tripsFilename, norm_mean=ten_year_growth_factor(1.0208), norm_sd=0, returnDataFrame=True, returnODinfo=True)
runtime208 = time.time()-start
print('Time elapsed: {:.2f}s'.format(runtime208))
print('Total demand: {}'.format(demand208))
print('        TSTT: {}'.format(tstt208))
print('Weighted V/C: {}'.format(wvc208))
df208.head()

Time elapsed: 1610.75s
Total demand: 853885.3223849998
        TSTT: 38900203.88628584
Weighted V/C: 7.108051502360516e-05


,flow,cost,der,capacity
link,,,,
"(719,1259)",18.928740,0.24,0.0,100000.0
"(1259,719)",1.228589,0.24,0.0,100000.0
"(909,1260)",8.600123,0.28,0.0,100000.0
"(1260,909)",45.457793,0.28,0.0,100000.0
"(705,1528)",212.545895,0.64,0.0,100000.0


## Scenario 2: 3% annual growth

In [5]:
start = time.time()
df3, tstt3, wvc3, demand3, od3 = tap_trial(
    netFilename, tripsFilename, norm_mean=ten_year_growth_factor(1.0208), norm_sd=0, returnDataFrame=True, returnODinfo=True)
runtime3 = time.time()-start
print('Time elapsed: {:.2f}s'.format(runtime3))
print('Total demand: {}'.format(demand3))
print('        TSTT: {}'.format(tstt3))
print('Weighted V/C: {}'.format(wvc3))
df3.head()

Time elapsed: 1585.60s
Total demand: 853885.3223849998
        TSTT: 38900203.88628584
Weighted V/C: 7.108051502360516e-05


,flow,cost,der,capacity
link,,,,
"(719,1259)",18.928740,0.24,0.0,100000.0
"(1259,719)",1.228589,0.24,0.0,100000.0
"(909,1260)",8.600123,0.28,0.0,100000.0
"(1260,909)",45.457793,0.28,0.0,100000.0
"(705,1528)",212.545895,0.64,0.0,100000.0


## Scenario 3: Geospatially adjusted growth

Suppose we had more precision in our estimates, allowing us to apply different growth factors to different sections of the city.

In [6]:
gfs = { # 5 different regions of the city correspond to three different scaling factors
    15: ten_year_growth_factor(1.0275), #west and central code to 1 and 5, respectively
    23: ten_year_growth_factor(1.0325), #south and north code to 2 and 3, respectively
    4: ten_year_growth_factor(1.03) #east codes to 4
}

df = pd.read_excel('inputs/ATX_PARTITIONED.xlsx', index_col=0)
df.head(50)

,X,Y,Field4
Node,,,
1,-97861763,30550045,1
2,-97873341,30595955,3
3,-97842574,30537706,3
4,-97967226,30531993,1
5,-97901257,30496550,1
6,-97882086,30460701,1
7,-97827879,30498174,1
8,-98083907,30434403,1
9,-98053644,30488843,1


In [7]:
west_central = set(df[(df['Field4']==1) | (df['Field4']==5)].index.tolist())
south_north = set(df[(df['Field4']==2) | (df['Field4']==3)].index.tolist())
east = set(df[(df['Field4']==4)].index.tolist())
assert(len(west_central) + len(south_north) + len(east) == df.shape[0])

In [9]:
start = time.time()
od_perturber(tripsFilename, 'normal', norm_mean=gfs[15], norm_sd=0, nodesPerturbedIfOrig=west_central)
od_perturber('trips_perturbed.tntp', 'normal', norm_mean=gfs[23], norm_sd=0, nodesPerturbedIfOrig=south_north)
dfr, tsttr, wvcr, demandr, odr = tap_trial(
    netFilename, 'trips_perturbed.tntp', norm_mean=gfs[4], norm_sd=0, nodesPerturbedIfOrig=east, returnDataFrame=True, returnODinfo=True)
runtimer = time.time()-start
print('Time elapsed: {:.2f}s'.format(runtimer))
print('Total demand: {}'.format(demand208))
print('        TSTT: {}'.format(tsttr))
print('Weighted V/C: {}'.format(wvcr))
dfr.head()

Time elapsed: 1649.22s
Total demand: 853885.3223849998
        TSTT: 53272535.40556267
Weighted V/C: 7.744013579425539e-05


,flow,cost,der,capacity
link,,,,
"(719,1259)",19.276519,0.24,0.0,100000.0
"(1259,719)",1.376894,0.24,0.0,100000.0
"(909,1260)",9.638260,0.28,0.0,100000.0
"(1260,909)",56.452666,0.28,0.0,100000.0
"(705,1528)",238.202712,0.64,0.0,100000.0
